# Laboratorium 11

## Wykorzystanie rozwiązania Kubernetes w przetwarzaniu danych masowych - K3S

Przypomnijmy diagram planowanego systemu::
![image](../data/project_flow_chart_L10.png)

W trakcie obecnego laboratorium rozpoczniemy przygotowania do Etapu IV.

Zapoznamy się dzisiaj z platformą Kubernetes oraz narzędziem Helm i zobaczymy w jaki sposób uruchamiać na Kubernetesie aplikacje.

### Kubernetes
Jest to platforma do zarządzania zasobami obliczeniowymi, utworzona przez Google'a. Kubernetes (K8s) zarządza kontenerami (np. Dockerowymi), przy czym najmniejszą jednostką operacyjną w K8s jest tzw. **pod**. Pod może składać się z wielu kontenerów i wolumenów dyskowych, jednak wybór liczby składowych poda jest zależny od konkretnego zastosowania. Najczęściej przyjmuje się, że jeden pod zawiera jedną aplikację - np. jeśli dana usługa zawsze wymaga własnej kopii innej usługi (np. narzędzia monitorującego) to warto te dwie usługi umieścić w jednym podzie. W naszych rozważaniach będziemy jednak przyjmować, że w jednym podzie zawsze będzie działać tylko jeden kontener oraz opcjonalnie jeden lub wiele wolumenów danych.


Pod względem architektury klaster Kubernetesowy składa się z węzłów typu:
- **master** - odpowiedzialne za zarządzanie klastrem, przyjmowanie nowych żądań od użytkownika, kontrolę uruchomionych podów itd.; w klastrze może działać wiele węzłów typu master, ale na nasze potrzeby wystarczy nam dokładnie jeden taki węzeł,
- **worker** - węzły, na których uruchamiają się właściwe zadania (pody); zazwyczaj klastry Kubernetesowe posiadają wiele (dziesiątki, setki) węzłów typu worker, ale w naszym przypadku uruchomimy tylko dwie instancje.

### Zadanie 11.1: przygotowanie infrastruktury (1 pkt)
Używając swojego konta na platformie AWS utwórz **co najmniej trzy** wirtualne maszyny EC2:
- użyj obrazu sytemu Ubuntu 18.04 (lub innego systemu z rodziny GNU/Linux, którego potrafisz używać)
- wybierz odpowiedni typ maszyn (ang. *flavour*) - co najmniej 4 vCPU oraz 4 GB RAMu

Jedna z maszyn będzie spełniała rolę mastera Kubernetesowego, a pozostałe (min. 2) maszyny będą działały w trybie worker.

### Zadanie 11.2: instalacja klastra K3s (7 pkt)
K3s jest uproszczoną wersją Kubernetesa, która jest znacząco łatwiejsza w instalacji. Wśród zasadnicznych zmian znajdziemy użycie bazy danych SQLite zamiast ETCD oraz ogranicznie funkcjonalności do minimum.

a) Zapoznaj się z dokumentacją K3s - [link](https://rancher.com/docs/k3s/latest/en/quick-start/)

b) Zainstaluj wszystkie wymagane przez K3s'a zależności na utworzonych maszynach wirtualnych (m.in. containerd oraz Docker)

c) Utwórz odpowiednie reguły firewalla na maszynach, tak aby komponenty K3s'a mogły się ze sobą komunikować.

d) Zgodnie z instrukcją zawartą w dokumentacji K3s'a zainstaluj klaster składający się z jednego węzła typu master oraz co najmniej dwóch węzłów typu worker. Upewnij się, że agenty K3s'a będą wykorzystywać Dockera.

e) Pobierz plik `kubeconfig` i zapisz go na lokalnej maszynie.

f) Zainstaluj narzędzie `kubectl` i sprawdź czy możesz się połączyć z utworzonym klastrem.


**Uwaga**: Do instalacji K3s'a możesz również wykorzystać narzędzie [k3sup](https://github.com/alexellis/k3sup)


Bonusowe zadania:
Wykorzystaj narzędzie Ansible i przygotuj playbooka, który zautomatyzuje proces instalacji klastra K3s na maszynach wirtualnych.

### Zadanie 11.3: Obiekty Kubernetesowe (9 pkt)
[Link do dokumentacji](https://kubernetes.io/docs/concepts/)

a) Wyjaśnij czym są i w jakim celu są używane następujące obiekty Kubernetesowe (4 pkt):
- Deployment
- Service (jakie mamy typy Service'ów i czym się różnią?)
- DaemonSet
- StatefulSet
- ConfigMap
- Secret
- PersistentVolume(Claim)

b) W jaki sposób możemy utworzyć te obiekty w klastrze (0.5 pkt)?

c) Wykorzystaj załączone w repozytorium (katalog: `plain_manifests`) manifesty Kubernetesowe (4.5 pkt):
- zainstaluj obiekty opisane przez manifesty na klastrze
- sprawdź czy wszystko zostało poprawnie zainstalowane
- wyjaśnij co każdy tych manifestów robi
- usuń (odinstaluj) uruchomione obiekty z klastra

Odpowiedzi do punktów a) i b) oraz komendy użyte w podpunkcie c) zapisz w pliku `odpowiedzi_3.md`.

### Zadanie 11.4: Helm (5 pkt)
Helm ([dokumentacja](https://helm.sh/docs/)) pozwala zautomatyzować proces uruchamiania aplikacji w klastrze Kubernetesowym (więcej o tym narzędziu na kolejnym wykładzie). Jest to menadżer paczek dla Kubernetesa – dostarcza wiele gotowych paczek dla typowych aplikacji (np. MongoDB, Redash), ale można również zdefiniować własne tzw. **charty** Helmowe dla naszych aplikacji.

Możemy zauważyć, że w manifestach K8s'owych wszystkie wartości (w tym nazwy) obiektów są zapisane na stałe. Jeśli byśmy chcieli uruchomić identyczny obiekt dla innej aplikacji w klastrze, musielibyśmy ręcznie zmienić jego nazwę, co w perspektywie bardziej złożonych aplikacji jest trudne i czasochłonne. Helm dostarcza silnik szablonów, który pozwala zamienić na stałe zapisane wartości w manifestach na szablony (ang. *template*), które są uzupełniane przez Helma w trakcie instalacji danego obiektu (zobacz poniżej).

Dodatkowo Helm wspiera i automatyzuje proces wdrażania, poprzez m.in. rolling updates oraz przywracanie zmian (*rollback*).


a) Zainstaluj narzędzie Helm CLI

b) Wykorzystaj załączony w repozytorium (katalog: `helm`) przykładowy chart Helmowy:
- zainstaluj go w klastrze
- sprawdź czy aplikacja działa i czy wszystkie obiekty zostały poprawnie zainstalowane,
- usuń chart z klastra

c) Wyjaśnij w jaki sposób wartości zapisane w pliku `values.yaml` są używane wewnątrz manifestów

d) W jaki sposób wykorzystać gotowe paczki (charty) Helmowe?


Odpowiedzi do punktów c) i d) oraz komendy używane w podpunkcie b) zapisz w pliku `odpowiedzi_4.md`